In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install spatialdata spatialdata_plot

In [ ]:
!pip install "anndata==0.12.2" "scanpy==1.11.4" "squidpy==1.6.5"

# Train on mouse brain data

This could take several hours. The trained model is provided, which can be used to try the interpretation part without training again.

In [4]:
import torch

# Check if CUDA is available
print("CUDA Available:", torch.cuda.is_available())

CUDA Available: True


In [5]:
##Working with google colab
import os
import sys

os.chdir("/content/drive/MyDrive/Thesis/Projects/Steamboat_X/examples")
cwd = os.getcwd()
print(cwd)

sys.path.append("../")
if torch.cuda.is_available():
    device = "cuda"
    print("GPU: ",torch.cuda.get_device_name(0))
else:
    device = "cpu"
    print("Using CPU")

/content/drive/MyDrive/Thesis/Projects/Steamboat_X/examples
GPU:  NVIDIA A100-SXM4-80GB


In [6]:
import scanpy as sc
import squidpy as sq
import pandas as pd
from tqdm.notebook import tqdm
import scipy as sp
import numpy as np
import multiprocessing
import pickle as pkl
import torch
import gc
import sklearn.metrics

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['mathtext.fontset'] = 'dejavuserif'
plt.rcParams['font.family'] = 'arial'

pltkw = dict(bbox_inches='tight', transparent=True)

/usr/lib/python3.12/importlib/__init__.py:90: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  return _bootstrap._gcd_import(name[level:], package, level)
/usr/local/lib/python3.12/dist-packages/anndata/__init__.py:44: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  return module_get_attr_redirect(attr_name, deprecated_mapping=_DEPRECATED)


In [7]:
import spatialdata as sd

In [8]:
import steamboat as sf
import steamboat.model as sm
#importlib.reload(sm)
#import steamboat.integrated_model
# importlib.reload(spaceformer.benchmarks)

In [9]:
import importlib
import steamboat.tools
import steamboat.model

In [10]:
Xenium_path = "/content/drive/MyDrive/Thesis/Projects/Data/Mouse_Brain_Coronal/data.zarr"
sdata = sd.read_zarr(Xenium_path)
sdata

/usr/local/lib/python3.12/dist-packages/zarr/creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/usr/local/lib/python3.12/dist-packages/zarr/creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/usr/local/lib/python3.12/dist-packages/zarr/creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/usr/local/lib/python3.12/dist-packages/zarr/creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/usr/local/lib/python3.12/dist-packages/zarr/creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/usr/local/lib/python3.12/dist-packages/zarr/creation.py:610: Use

SpatialData object, with associated Zarr store: /content/drive/MyDrive/Thesis/Projects/Data/Mouse_Brain_Coronal/data.zarr
├── Images
│     ├── 'he_image': DataTree[cyx] (3, 24689, 17051), (3, 12344, 8525), (3, 6172, 4262), (3, 3086, 2131), (3, 1543, 1065)
│     └── 'morphology_focus': DataTree[cyx] (4, 23912, 34154), (4, 11956, 17077), (4, 5978, 8538), (4, 2989, 4269), (4, 1494, 2134)
├── Labels
│     ├── 'cell_labels': DataTree[yx] (23912, 34154), (11956, 17077), (5978, 8538), (2989, 4269), (1494, 2134)
│     └── 'nucleus_labels': DataTree[yx] (23912, 34154), (11956, 17077), (5978, 8538), (2989, 4269), (1494, 2134)
├── Points
│     └── 'transcripts': DataFrame with shape: (<Delayed>, 13) (3D points)
├── Shapes
│     ├── 'cell_boundaries': GeoDataFrame shape: (63173, 1) (2D shapes)
│     ├── 'cell_circles': GeoDataFrame shape: (63173, 2) (2D shapes)
│     └── 'nucleus_boundaries': GeoDataFrame shape: (63036, 1) (2D shapes)
└── Tables
      └── 'table': AnnData (63173, 5006)
with coordi

In [11]:
adata = sdata.tables["table"]
adata

AnnData object with n_obs × n_vars = 63173 × 5006
    obs: 'cell_id', 'transcript_counts', 'control_probe_counts', 'genomic_control_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'nucleus_count', 'segmentation_method', 'region', 'z_level', 'cell_labels'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatialdata_attrs'
    obsm: 'spatial'

In [ ]:
####SINA
# to see what's inside:

adata.obs.head()
#adata.obs.columns
#adata.var["gene_symbol"].unique().shape
#adata.var["gene_symbol"].to_list()[:20]

,cell_id,transcript_counts,control_probe_counts,genomic_control_counts,control_codeword_counts,unassigned_codeword_counts,deprecated_codeword_counts,total_counts,cell_area,nucleus_area,nucleus_count,segmentation_method,region,z_level,cell_labels
0,aaaalabd-1,2193,1,0,0,3,2,2199,49.626721,33.099532,1.0,Segmented by boundary stain (ATP1A1+CD45+E-Cad...,cell_circles,0.0,1
1,aaabcnhh-1,1873,0,0,0,0,0,1873,55.226096,38.427970,1.0,Segmented by boundary stain (ATP1A1+CD45+E-Cad...,cell_circles,0.0,2
2,aaacnmip-1,2163,0,0,1,1,0,2165,85.796878,75.456096,1.0,Segmented by boundary stain (ATP1A1+CD45+E-Cad...,cell_circles,0.0,3
3,aaadikih-1,2441,0,0,0,4,1,2446,94.060472,52.290939,1.0,Segmented by boundary stain (ATP1A1+CD45+E-Cad...,cell_circles,0.0,4
4,aaaeinek-1,1407,0,0,0,0,0,1407,112.258442,NaN,0.0,Segmented by boundary stain (ATP1A1+CD45+E-Cad...,cell_circles,0.0,5


In [ ]:
####SINA
adata.var_names.to_list()[-5:]

['Zswim9', 'Zup1', 'Zyx', 'Zzef1', 'a']

In [ ]:
####SINA

adata.X[20:30, -5:].toarray()

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 1., 0., 1., 0.]], dtype=float32)

In [ ]:
####SINA

adata.shape      # (cells, genes)
#adata.n_vars

(63173, 5006)

In [ ]:
####SINA

adata.uns['spatialdata_attrs']

{'instance_key': 'cell_id', 'region': 'cell_circles', 'region_key': 'region'}

In [ ]:
####SINA
for key in adata.obs.keys():
    print(f"{key}: ", "\n", adata.obs[key], "\n", "Unique Elements: ", adata.obs[key].unique().shape, "\n")

cell_id:  
 0        aaaalabd-1
1        aaabcnhh-1
2        aaacnmip-1
3        aaadikih-1
4        aaaeinek-1
            ...    
63168    oildnokf-1
63169    oildpboo-1
63170    oilgaepb-1
63171    oilgmbbn-1
63172    oilhbhom-1
Name: cell_id, Length: 63173, dtype: object 
 Unique Elements:  (63173,) 

transcript_counts:  
 0        2193
1        1873
2        2163
3        2441
4        1407
         ... 
63168     112
63169     663
63170     320
63171     299
63172     557
Name: transcript_counts, Length: 63173, dtype: int64 
 Unique Elements:  (4815,) 

control_probe_counts:  
 0        1
1        0
2        0
3        0
4        0
        ..
63168    0
63169    0
63170    0
63171    0
63172    0
Name: control_probe_counts, Length: 63173, dtype: int64 
 Unique Elements:  (3,) 

genomic_control_counts:  
 0        0
1        0
2        0
3        0
4        0
        ..
63168    0
63169    0
63170    0
63171    0
63172    0
Name: genomic_control_counts, Length: 63173, dtype: int64

In [12]:
if False:
    n_genes = 100
    np.random.seed(0)
    chosen_gene_mask = np.zeros(adata.shape[1])
    chosen_gene_mask[:n_genes] = 1.
    np.random.shuffle(chosen_gene_mask)
    chosen_gene_mask = chosen_gene_mask > 0.
    adata = adata[:, chosen_gene_mask]

# Normalizing to median total counts
sc.pp.normalize_total(adata)
# Logarithmize the data
sc.pp.log1p(adata)

/usr/local/lib/python3.12/dist-packages/legacy_api_wrap/__init__.py:88: UserWarning: Some cells have zero counts
  return fn(*args_all, **kw)


In [13]:
adatas = []
for i in adata.obs['region'].unique():
    adatas.append(adata[adata.obs['region'] == i])
    adatas[-1].obs['global'] = 0  #Only support one unique value for regional observation.

/tmp/ipython-input-4759537.py:4: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adatas[-1].obs['global'] = 0  #Only support one unique value for regional observation.


In [14]:
adatas = sf.prep_adatas(adatas, norm=False, log1p=False)

  0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
dataset = sf.make_dataset(adatas, sparse_graph=True, regional_obs=['global'])

Using ['global'] as regional annotations.


  0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
sf.set_random_seed(0)
model = sm.Steamboat(adata.var_names.tolist(), n_heads=50, n_scales=3)
model = model.to(device)
# model.load_state_dict(torch.load('saved_models/mmbrain_new.pth', weights_only=True))


In [17]:
model.fit(dataset, entry_masking_rate=0.3, feature_masking_rate=0.3,
          device=device,
          max_epoch=10000,
          loss_fun=torch.nn.MSELoss(reduction='sum'),
          opt=torch.optim.Adam,
          sched=torch.optim.lr_scheduler.OneCycleLR,
          max_lr=0.1, opt_args=dict(lr=0.1), stop_eps=1e-3, report_per=200, stop_tol=200)

[2025-11-04 05:13:13,763::train::INFO] Epoch 1: train_loss 540.73407
INFO:train:Epoch 1: train_loss 540.73407
[2025-11-04 05:14:54,832::train::INFO] Epoch 201: train_loss 457.28519
INFO:train:Epoch 201: train_loss 457.28519
[2025-11-04 05:16:34,794::train::INFO] Epoch 401: train_loss 355.94489
INFO:train:Epoch 401: train_loss 355.94489
[2025-11-04 05:18:14,656::train::INFO] Epoch 601: train_loss 324.34991
INFO:train:Epoch 601: train_loss 324.34991
[2025-11-04 05:19:54,500::train::INFO] Epoch 801: train_loss 314.90979
INFO:train:Epoch 801: train_loss 314.90979
[2025-11-04 05:21:34,530::train::INFO] Epoch 1001: train_loss 312.33386
INFO:train:Epoch 1001: train_loss 312.33386
[2025-11-04 05:23:14,601::train::INFO] Epoch 1201: train_loss 310.89868
INFO:train:Epoch 1201: train_loss 310.89868
[2025-11-04 05:24:54,867::train::INFO] Epoch 1401: train_loss 309.43027
INFO:train:Epoch 1401: train_loss 309.43027
[2025-11-04 05:26:35,187::train::INFO] Epoch 1601: train_loss 308.71033
INFO:train:Epo

Steamboat(
  (spatial_gather): BilinearAttention(
    (bias): NonNegBias(
      (elu): ELU(alpha=1.0)
    )
    (q): NonNegLinear(
      (elu): ELU(alpha=1.0)
    )
    (k_local): NonNegLinear(
      (elu): ELU(alpha=1.0)
    )
    (k_regionals): ModuleList(
      (0): NonNegLinear(
        (elu): ELU(alpha=1.0)
      )
    )
    (w_ego): NonNegScale(
      (elu): ELU(alpha=1.0)
    )
    (w_local): NonNegScale3(
      (elu): ELU(alpha=1.0)
    )
    (w_global): NonNegScale3(
      (elu): ELU(alpha=1.0)
    )
    (tanh): Tanh()
    (v): NonNegLinear(
      (elu): ELU(alpha=1.0)
    )
    (cosine_similarity): CosineSimilarity()
  )
)

In [ ]:
"""
Input and Output of the model:

Input:

Graph structure (adj_list, regional_adj_lists)

Cell features (x, masked_x)

Regional features (regional_xs)

Output:

Reconstructed cell features ([n_cells, n_genes])

attention weights + embeddings (dict)

"""


In [18]:
##working with google colab
if True:
    torch.save(model.state_dict(), 'saved_models/Xmbrain_new_50_0.3.pth')
else:
    model.load_state_dict(torch.load('saved_models/Xmbrain.pth',weights_only=True))

In [ ]:
##working with laptop
if False:
    torch.save(model.state_dict(), 'saved_models/mmbrain_new.pth')
else:
    model.load_state_dict(torch.load('saved_models/mmbrain.pth', map_location=torch.device('cpu'),weights_only=True))

In [19]:
## getting the attention scores fore each cell

def annotate_adatas(adatas: list[sc.AnnData], dataset, model,
                    device=device, get_recon=False):
    """_summary_

    :param adatas: _description_
    :param dataset: _description_
    :param model: _description_
    :param device: _description_, defaults to 'cuda'
    """
    # Safeguards
    assert len(adatas) == len(dataset), "mismatch in lenghths of adatas and dataset"
    for adata, data in zip(adatas, dataset):
        assert adata.shape[0] == data[0].shape[0], f"adata[{i}] has {adata.shape[0]} cells but dataset[{i}] has {data[0].shape[0]}."

    for i, (x, adj_list, regional_xs, regional_adj_lists) in tqdm(enumerate(dataset), total=len(dataset)):
        adj_list = adj_list.squeeze(0).to(device)
        x = x.squeeze(0).to(device)
        regional_adj_lists = [regional_adj_list.to(device) for regional_adj_list in regional_adj_lists]
        regional_xs = [regional_x.to(device) for regional_x in regional_xs]

        with torch.no_grad():
            res, details = model(adj_list, x, x, regional_adj_lists, regional_xs, get_details=True)

            if get_recon:
                adatas[i].obsm['X_recon'] = res.cpu().numpy()

            adatas[i].obsm['q_score'] = details['embq'].cpu().numpy()

            adatas[i].obsm['k_score'] = details['embk'][0].cpu().numpy()
            adatas[i].obsm['k_score_global'] = model.spatial_gather.k_regionals[0](x).cpu().numpy()
            for j, emb in enumerate(details['embk'][1]):
                adatas[i].uns[f'embk_{j}'] = emb.cpu().numpy()
            adatas[i].obsm['attn'] = details['attn'].cpu().numpy()
            adatas[i].obsm['ego_attn'] = details['attnm'][0].cpu().numpy()
            adatas[i].obsm['local_attn'] = details['attnm'][1].cpu().numpy()
            for j, matrix in enumerate(details['attnm'][2]):
                adatas[i].obsm[f'regional_attn_{j}'] = matrix.cpu().numpy()

            # local attention (as graph)
            for j in range(model.spatial_gather.n_heads):
                w = details['attnp'][1].cpu().numpy()[:, j, :].flatten()
                # print(details['attnp'][1].cpu().shape)
                # print(len(w))
                uv = adj_list.cpu().numpy()
                u = uv[0]
                v = uv[1]
                # print(len(u))
                # print(len(v))
                if uv.shape[0] == 3: # masked for unequal neighbors
                    m = (uv[2] > 0)
                    w, u, v = w[m], u[m], v[m]
                adatas[i].obsp[f'local_attn_{j}'] = sp.sparse.csr_matrix((w, (u, v)),
                                                                            shape=(adatas[i].shape[0],
                                                                                adatas[i].shape[0]))

In [20]:
annotate_adatas(adatas, dataset, model, get_recon=True)

  0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
v_weights = model.spatial_gather.v.weight.detach().cpu().numpy().sum(axis=0)
adata.uns['steamboat_v_weights'] = v_weights

In [22]:
## Saving the scores:
def gather_obs(adata, adatas):
    all_embq = []
    all_embk = []
    all_embk_glb = []
    all_ego_attn = []
    all_local_attn = []
    all_global_attn = []
    all_attn = []
    all_recon = []
    for i in range(len(adatas)):
        all_embq.append(adatas[i].obsm['q_score'])
        all_embk.append(adatas[i].obsm['k_score'])
        all_ego_attn.append(adatas[i].obsm['ego_attn'])
        all_local_attn.append(adatas[i].obsm['local_attn'])
        all_global_attn.append(adatas[i].obsm['regional_attn_0'])
        all_recon.append(adatas[i].obsm['X_recon'])
        all_attn.append(adatas[i].obsm['attn'])
        all_embk_glb.append(adatas[i].obsm['k_score_global'])
    adata.obsm['q_score'] = np.vstack(all_embq)
    adata.obsm['k_score'] = np.vstack(all_embk)
    adata.obsm['ego_attn'] = np.vstack(all_ego_attn)
    adata.obsm['local_attn'] = np.vstack(all_local_attn)
    adata.obsm['global_attn'] = np.vstack(all_global_attn)
    adata.obsm['X_recon'] = np.vstack(all_recon)
    adata.obsm['attn'] = np.vstack(all_attn)
    adata.obsm['k_score_global'] = np.vstack(all_embk_glb)

In [23]:
gather_obs(adata, adatas)

In [24]:
os.makedirs("saved_h5ad", exist_ok=True)
adata.write_h5ad("saved_h5ad/Xmbrain_50_0.3.h5ad")

In [25]:
for i, adata in enumerate(tqdm(adatas)):
    adata.write_h5ad(f"saved_h5ad/Xmbrain_{i}_50_0.3.h5ad")

  0%|          | 0/1 [00:00<?, ?it/s]